# Capítulo 16: Natural language processing with RNNs and attention (página 525)

Una aproximación común al realizar NLP es utilizar redes neuronales recurrentes (RNN)
En este primer ejercicio haremos unpredictor de caracteres (qué caracter iría después de otro probablemente)
RNN stateless --> Aprende de posiciones aleatorias de texto en cada iteración sin información alguna del resto del texto.
RNN stateful --> Que preserva un "estado" oculto a través de las iteraciones de entrenamiento y continúa leyendo donde lo dejó previamente.

# Importando nuestros paquetes

In [12]:
from tensorflow import keras
import numpy as np

# Creando el dataset de entrenamiento

Primero, descargamos el dataset del trabajo de Shakespeare's utilizando Keras

In [3]:
shakespeare_url = "https://homl.info/shakespeare" # URL recortada
filepath = keras.utils.get_file('Shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 1s 1us/step


Ahora que ya tenemos el texto cargado en memoria, necesitamos tokenizarlo, esto significa que daremos un valor numérico a cada caracter único presente en nuestro set de datos. Para ello, utilizaremos Tokenizer de Keras.

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

Establecemos `char_level=True` para obtener los tokens a nivel de caracter y no de palabra, como lo hace ya por defecto la función.
Nota que tokenizer convierte el texto a minúsculas por defecto, pero puedes utilizar el argumento `lower=True` para evitar este comportamiento.

Ahora, el tokenizador puede codificar un enunciado (o una lista de enunciados) a una lista de IDs de caracteres y viceversa, y entonces nos puede decir que tantos caracteres distintos hay y el total de caracteres distintos en el texto.

## Ejemplo del uso de tokenizer:

In [6]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) # total de caracteres distintos
max_id

39

In [10]:
dataset_size = tokenizer.document_count # total de caracteres en el documento
dataset_size

1115394

Vamos a codificar el texto completo así, cada caracter es representado por si propio ID (restaremos 1 a cada ID para obtener IDs del 0 al 38 en lugar del 1 al 39):

In [14]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
encoded

array([19,  5,  8, ..., 20, 26, 10])